# 삼성전자의 손익계산서, 재무상태표 정보 출력 및 활용
### 법인등록번호 130111-0006246
### 사업자등록번호 124-81-00998
### 회계연도 2020

# ------------------------------
### tool: requests
#### $ python -m pip install requests
### tool: xmltodict
#### $ pip install xmltodict
# ------------------------------
### 참고 : https://www.geeksforgeeks.org/python-xml-to-json/ (XML -> JSON 변환법)


In [1]:
import requests as rq
import json, xmltodict

ServiceKey = "ZS%2Fc%2B16anKABmvp0K8KuNoC3%2Fuwb0luEMbFLqCfjFxRuvQGeh2a5VWSgI7acEMi0A27cn%2BRwDQfCOSm5t%2Bs1PQ%3D%3D"

CorpNum = "130111" + "0006246"

Year = "2020"

#손익계산서 URL
S_I = "http://apis.data.go.kr/1160100/service/GetFinaStatInfoService/getIncoStat?serviceKey="+ServiceKey+"&numOfRows=10&pageNo=1&numOfRows=xml&crno="+CorpNum+"&bizYear="+Year
#재무상태표 URL
S_FP = ""

ModuleNotFoundError: No module named 'requests'

In [10]:
S_I_content = rq.get(S_I).content
S_I_dict = xmltodict.parse(S_I_content)

NameError: name 'rq' is not defined

In [11]:
#string 형태의 json
S_I_jsonString = json.dumps(S_I_dict['response']['body']['items']['item'], ensure_ascii=False)

NameError: name 'json' is not defined

In [12]:
#string -> json
S_I_jsonObj = json.loads(S_I_jsonString)

NameError: name 'json' is not defined

In [13]:
#영미식 단위로 끊어주는 함수

def money(mon):
    a = mon
    result = str(mon)
    finalresult = ""
    count = 0
    while (a > 0):
        a = a//1000
        if(a != 0):
            count+=1
    for i in range(count):
        info = -3*count
        index = info+i*3
        result = result[:index] + ',' + result[index:]
        
    return result
#----------------
#test
money(1320000)
#result = 1,320,000
#----------------
# O(n)

'1,320,000'

In [14]:
for i in S_I_jsonObj:
    print('====',i['acitNm'],'====')
    print("당분기:", money(int(i['thqrAcitAmt']))+"원")
    print("당기:",money(int(i['crtmAcitAmt']))+"원")
    print("전분기:",money(int(i['lsqtAcitAmt']))+"원")
    print("전기:",money(int(i['pvtrAcitAmt']))+"원")
    print("전전기:",money(int(i['bpvtrAcitAmt']))+"원")
    if(i['fnclDcdNm'] == "연결재무제표 [member]"):
        print("재무제표 종류:","연결재무제표\n")
    elif i['fnclDcdNm'] == "별도재무제표 [member]":
        print("재무제표 종류:","별도재무제표\n")

NameError: name 'S_I_jsonObj' is not defined